#### REGULARIZATION

We want to create two prediction models both for solar energy output and wind energy output. In order to predict these outputs we want to build a model that selects the best possible predictors maximizing the feature significance, avoiding colinearity, ensuring sparisty, considering the best feature transformations... To do so we will apply different models like Lasso, and Ridge and then we can apply Holistic Regression to see how the model is improved by it. We will run all of these models by all predictors of solar and energy output. 

In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames
#read train and validation data and split it into X and y
X_train_solar=Matrix(DataFrame(CSV.File("X_train_solar.csv")))
y_train_solar=Matrix(DataFrame(CSV.File("y_train_solar.csv")))
X_train_wind=Matrix(DataFrame(CSV.File("X_train_wind.csv")))
y_train_wind=Matrix(DataFrame(CSV.File("y_train_wind.csv")))

X_valid_solar=Matrix(DataFrame(CSV.File("X_valid_solar.csv")))
y_valid_solar=Matrix(DataFrame(CSV.File("y_valid_solar.csv")))
X_valid_wind=Matrix(DataFrame(CSV.File("X_valid_wind.csv")))
y_valid_wind=Matrix(DataFrame(CSV.File("y_valid_wind.csv")));

### Functions

In [2]:
function lasso(X,y,lambda=0.25)
    
    #Build model
    model = Model(Gurobi.Optimizer)#we have defined the model, pass Gurobi optimizer into the model
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    n, p = size(X)
    
    @variable(model, beta[j=1:p])
    @variable(model, beta_abs[j=1:p])
    
    
    #Insert constraints

    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
    
    #sum((y-X*beta).^2)) = sum((y[i] - sum(X[i,j]*beta[j]) for j=1:p)^2 for i=1:n)
    
    #Insert objective
    @objective(model, Min, sum((y-X*beta).^2) + lambda*sum(beta_abs))
    
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
    
end

lasso (generic function with 2 methods)

In [3]:
function mse(X,y, beta)
    return sum((y-X*beta).^2)/length(y)
end

mse (generic function with 1 method)

In [4]:
function correlation(X,p)
    correlation_matrix = Statistics.cor(X) #compute corr matrix
    n,m = size(correlation_matrix) #set sizes
    correlated_pairs=[] #empty list to store correlated pairs
    for i=1:n #for all features
        for j=i+1:m 
            if abs(correlation_matrix[i,j])>p #if the abs value of the corr is higher than p
                push!(correlated_pairs, (i,j)) #append pair to list of correlated pairs
            end
        end
    end
    return correlated_pairs
end

correlation (generic function with 1 method)

In [5]:
#Build transformation function
function transformation(X)
    X_old=DataFrame(X, :auto) #define X as a df
    X_new=DataFrame() #new empty df 
    n,p=size(X_old)
    e=1
    for i=1:p #for each feature in X add 4 transformations
        X_new[!, "X$i"]=X_old[:,i] #transformation 1: no transformation
        X_new[!, "Sqrt$i"]=X_old[:,i].^2 #transformation 2: square root
        #X_new[!, "Log$i"]=log.(abs.(X_old[:,i])) #transformation 3: log
        X_new[!, "Abs_Sqrt$i"]=sqrt.(abs.(X_old[:,i])) #transformation 3: absolute squared root
        #square root of absolute value
        
    end
    return(X_new) #we return a new df with all transformations, it will have size nxp*4
end

transformation (generic function with 1 method)

In [6]:
function holistic_regression(X,y,lambda=0.25, per=0.6, M=50, k=10) #add parameter per
    
    #Call functions
    X_new=Matrix(transformation(X)) #call function with all transformations of X
    HC = correlation(X_new, per) #call correlation function to compute hc_pairs
    
    #Set sizes
    n,p_new=size(X_new)
    
    #Build model
    model = Model(Gurobi.Optimizer)
    #model = Model(Gurobi.Optimizer, NonConvex = 2)#we have defined the model, pass Gurobi optimizer into the model
    #model = Model(with_optimizer(Gurobi.Optimizer, NonConvex = 2))
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    @variable(model, beta[1:p_new])
    @variable(model, beta_abs[1:p_new])
    @variable(model, z[1:p_new], Bin) #we add a binary variable
    

    #Insert constraints
    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
        
    #sparsity constraint: over all 60 features (including transformations) 
    @constraint(model, -M*z .<= beta)
    @constraint(model, beta .<= M*z)
    @constraint(model, sum(z) <= k) 
    
    #constraint on Transformation: from the 4 transformations per each feature we only select one
    for i=1:3:p_new
        #for j=i:i+3 #for every 4 transformations
        @constraint(model, sum(z[i:i+2])<=1)
        #end
    end #we get a vector with 15 features
    
    #constraint on HC pairs once we have selected 15 features 
    for (i,j) in HC
        @constraint(model, z[i] + z[j] <= 1)
    end #we can only take one of the pairs of correlated pairs
    #@constraint(model, sum(z[i])<=k) #ensure that the model has at most 8 features
    
        
    #Insert objective
    @objective(model, Min, sum((y-X_new*beta).^2) + lambda*sum(beta_abs))
    
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
    
end

holistic_regression (generic function with 5 methods)

### NORMALIZE DATA
We normalize the data with min max scaling. We apply normalization for the numerical features only and then we use alpha to balance between numerical and binary variables.

In [7]:
function min_max_scaling(X, num_feature_indices)
    for i in num_feature_indices
        X[:, i] = (X[:, i] .- minimum(X[:, i])) ./ (maximum(X[:, i]) .- minimum(X[:, i]))
    end
    return X
end

min_max_scaling (generic function with 1 method)

#### SOLAR

In [8]:
#scale SOLAR TRAIN only numerical
X_train_solar=min_max_scaling(Matrix(X_train_solar), 1:15)

#numerical features
X_num_solar = X_train_solar[:, 1:15]

# Binary features as they are
X_bin_solar = X_train_solar[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_solar = alpha*X_num_solar
X_bin_solar = (1-alpha)*X_bin_solar;

# Append data
X_train_solar_norm = [X_num_solar X_bin_solar];

In [9]:
#scale SOLAR TRAIN only numerical
X_valid_solar=min_max_scaling(Matrix(X_valid_solar), 1:15)

#numerical features
X_num_solar = X_valid_solar[:, 1:15]

# Binary features as they are
X_bin_solar = X_valid_solar[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_solar = alpha*X_num_solar
X_bin_solar = (1-alpha)*X_bin_solar;

# Append data
X_valid_solar_norm = [X_num_solar X_bin_solar];

In [10]:
#scale Y 
y_train_solar_norm = min_max_scaling(Matrix(y_train_solar), 1:1)
y_valid_solar_norm = min_max_scaling(Matrix(y_valid_solar), 1:1);

#### WIND

In [11]:
X_train_wind=min_max_scaling(Matrix(X_train_wind), 1:15)

#numerical features
X_num_wind = X_train_wind[:, 1:15]

# Binary features as they are
X_bin_wind = X_train_wind[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_wind = alpha*X_num_wind
X_bin_wind = (1-alpha)*X_bin_wind;

# Append data
X_train_wind_norm = [X_num_wind X_bin_wind];

In [12]:
X_valid_wind=min_max_scaling(Matrix(X_valid_wind), 1:15)

#numerical features
X_num_wind = X_valid_wind[:, 1:15]

# Binary features as they are
X_bin_wind = X_valid_wind[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_wind = alpha*X_num_wind
X_bin_wind = (1-alpha)*X_bin_wind;

# Append data
X_valid_wind_norm = [X_num_wind X_bin_wind];

In [13]:
#scale Y 
y_train_wind_norm=min_max_scaling(Matrix(y_train_wind), 1:1)
y_valid_wind_norm=min_max_scaling(Matrix(y_valid_wind), 1:1);

### SOLAR ENERGY OUTPUT

* Lasso: Most values tending to 0, MSE of 0.00673 
* Holistic Regression: do CV to get optimal value of k, rho, and lambda. (Best lambda: 0.9 , Best rho: 0.6, Best k: 7)
    - Selected features (7): Square root of FEELSLIKE, Square root of HUMIDITY, Squared of SNOWDEPTH, Square root of WINDSPEED, Squared of WINDDIR, Square root of SEALEVELPRESSURE, and Square root of SOLARIRRADIATION
    - MSE: 0.0065948

In [15]:
beta_lasso=lasso(X_train_solar_norm,y_train_solar_norm,0.25)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


26-element Vector{Float64}:
  0.06870883738907983
  1.9582063922296577e-5
 -1.1596794512198694e-9
  0.040820647909880534
  1.0539578082266574e-7
  0.1474800066324126
 -1.6836327599483506e-10
  0.08055316093521973
  0.044749045831679386
  0.024747523268956784
  0.06862996744977973
  0.00772413078671385
 -0.006102740436276708
  0.12941353118330381
  0.04487727403013623
 -0.005411236131922988
  0.021525307063992877
  0.017723833922615135
 -1.1743351130647954e-9
  0.009540994338291324
 -0.02486523996193092
  0.04805912355482118
  0.426863525745457
  3.6693832470317106e-10
  0.04805912355482118
 -0.024865240037085963

In [16]:
mse(X_valid_solar_norm, y_valid_solar_norm, beta_lasso)

0.006732678993053726

In [17]:
beta_opt=holistic_regression(X_train_solar_norm, y_train_solar_norm, 0.2, 0.6, 50, 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


78-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.07629703379779054
 0.0
 0.0
 0.0
 0.0
 0.06429750984401607
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [18]:
#report mse
X_valid_solar_norm_trans=Matrix(transformation(X_valid_solar_norm))
mse(X_valid_solar_norm_trans, y_valid_solar_norm, beta_opt)

0.006753220086073494

#### Cross validation

In [19]:
#find best lambda, rho, and k
lowest_mse=Inf
best_lambda=0
best_rho=0
best_k=0
for lambda in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    for rho in [0.5, 0.6, 0.7, 0.8, 0.9]
        for k in [6,7,8,9,10,11,12,13]
            beta_opt=holistic_regression(X_train_solar_norm, y_train_solar_norm, lambda, rho, 50, k)
            X_valid_solar_norm_trans=Matrix(transformation(X_valid_solar_norm))
            mse_temp= mse(X_valid_solar_norm_trans, y_valid_solar_norm, beta_opt)
            #for each lambda, find the best rho
            if mse_temp<lowest_mse
                lowest_mse=mse_temp
                best_lambda=lambda
                best_rho=rho
                best_k=k
            end
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19

In [20]:
println("Best lambda: ", best_lambda)
println("Best rho: ", best_rho)
println("Best k: ", best_k)

Best lambda: 0.9
Best rho: 0.6
Best k: 7


In [21]:
beta_opt=holistic_regression(X_train_solar_norm, y_train_solar_norm, best_lambda, best_rho, 50, best_k)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


78-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.07396910492719036
 0.0
 0.0
 0.0
 0.0
 0.0
 0.07174690884748164
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [22]:
#return indices of selected features
selected_features=findall(x->x>0, beta_opt)

7-element Vector{Int64}:
  6
 12
 23
 27
 29
 33
 42

In [23]:
#what is the name of the selected features
names(transformation(X_train_solar))[selected_features]

7-element Vector{String}:
 "Abs_Sqrt2"
 "Abs_Sqrt4"
 "Sqrt8"
 "Abs_Sqrt9"
 "Sqrt10"
 "Abs_Sqrt11"
 "Abs_Sqrt14"

* Square root of FEELSLIKE
* Square root of HUMIDITY
* Squared of SNOWDEPTH
* Square root of WINDSPEED
* Squared of WINDDIR
* Square root of SEALEVELPRESSURE
* Square root of SOLARIRRADIATION

In [24]:
#report mse
X_valid_solar_norm_trans=Matrix(transformation(X_valid_solar_norm))
mse(X_valid_solar_norm_trans, y_valid_solar_norm, beta_opt)

0.006594836542325529

### WIND ENERGY OUTPUT
* Lasso: Most values tending to 0. MSE of 0.04095
* Holistic Regression: do CV to get optimal value of k, rho, and lambda. (Best lambda: 0.9, Best rho: 0.9, Best k: 10)
    - Selected features (5): Square root of DEW, Square root of PRECIPITATION PROBABILITY, WINDSPEED, Square root of WINDDIRECTION, and Squared of VISIBILITY
    - MSE of 0.038496

In [25]:
beta_lasso=lasso(X_train_wind_norm,y_train_wind_norm,0.25)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


26-element Vector{Float64}:
 -0.9270558512228663
 -3.234600593034686e-7
  0.7098056676542894
 -0.5162594121303907
  0.07115161770843202
  0.5674761124290322
 -5.712486193068013e-12
 -0.11878826773037546
  2.073914966456901
  0.1030496405496948
 -0.23215552292312852
 -0.07006931359906586
  0.08253860171156757
 -0.2694265522480585
 -8.522801299286038e-12
  0.06117491919084512
 -0.006265716142169834
  0.05407229314215172
  0.053735586247040536
 -3.058133904148284e-13
  8.214914232075702e-12
 -0.021470004286566192
  1.5705799135681962
  3.1607236467191433e-8
 -0.021470004286566192
  8.214914232075702e-12

In [26]:
mse(X_valid_wind_norm, y_valid_wind_norm, beta_lasso)

0.04094678546061689

### CROSS VALIDATION: best lambda, rho, and k

In [27]:
#find best lambda
lowest_mse=Inf
best_lambda=0
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, i, 0.6, 50, 10)
    X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
    mse_temp= mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)
    if mse_temp<lowest_mse
        lowest_mse=mse_temp
        best_lambda=i
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


In [28]:
#find best lambda and best rho
lowest_mse=Inf
best_lambda=0
best_rho=0
for lambda in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    for rho in [0.5, 0.6, 0.7, 0.8, 0.9]
        beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, lambda, rho, 50, 10)
        X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
        mse_temp= mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)
        #for each lambda, find the best rho
        if mse_temp<lowest_mse
            lowest_mse=mse_temp
            best_lambda=lambda
            best_rho=rho
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19

In [29]:
#find best lambda, rho, and k
lowest_mse=Inf
best_lambda=0
best_rho=0
best_k=0
for lambda in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    for rho in [0.5, 0.6, 0.7, 0.8, 0.9]
        for k in [6,7,8,9,10,11,12,13]
            beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, lambda, rho, 50, k)
            X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
            mse_temp= mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)
            #for each lambda, find the best rho
            if mse_temp<lowest_mse
                lowest_mse=mse_temp
                best_lambda=lambda
                best_rho=rho
                best_k=k
            end
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19

In [30]:
println("Best lambda: ", best_lambda)
println("Best rho: ", best_rho)
println("Best k: ", best_k)

Best lambda: 0.9
Best rho: 0.9
Best k: 10


In [31]:
#holistic regression
beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, 0.1, 0.45, 50, 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


78-element Vector{Float64}:
  0.0
 -0.17521442134658463
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.1806348401826435
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [34]:
#mse
X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)

0.043026241287980466

In [35]:
#holistic regression with optimal hyperparameters
beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, best_lambda, best_rho, 50, best_k)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


78-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
 -0.4525511538192362
  0.0
  0.0
  0.0
  0.30649204226385673
  0.0
 -0.3187568928597829
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [36]:
#mse
X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt) #improve mse

0.03849602906347231

In [37]:
#return indices of selected features
selected_features=findall(x->x>0, beta_opt)

5-element Vector{Int64}:
  9
 18
 25
 30
 38

In [38]:
#what is the name of the features?
names(transformation(X_train_wind))[selected_features]

5-element Vector{String}:
 "Abs_Sqrt3"
 "Abs_Sqrt6"
 "X9"
 "Abs_Sqrt10"
 "Sqrt13"

* Square root of DEW
* Square root of PRECIPITATION PROBABILITY
* WINDSPEED
* Square root of WINDDIRECTION
* Squared of VISIBILITY